# ☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda

Este notebook introduce un flujo moderno de datos en AWS con almacenamiento en S3, transformación con Glue (PySpark), consulta con Athena (SQL sobre S3) y orquestación con eventos/Lambda. Incluye ejemplos de código con `boto3` y prácticas recomendadas.

## Requisitos y Notas de Ejecución

- Para ejecutar código real de AWS necesitas credenciales configuradas en tu entorno (`AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION`).
- Nunca subas credenciales al repositorio. Usa variables de entorno o perfiles de AWS CLI.
- Este notebook es auto-contenido con bloques que pueden ejecutarse si tu entorno ya tiene permisos.
- Alternativa local: usar LocalStack para simular servicios AWS en tu máquina.

### ☁️ **AWS Cloud: Ecosistema para Data Engineering**

**Stack Moderno de AWS para Datos:**

1. **S3 (Simple Storage Service)**: Data Lake base
   - Almacenamiento ilimitado con 99.999999999% durabilidad (11 nines)
   - Pricing: Pay-per-use (~$0.023/GB/mes en Standard)
   - Clases de almacenamiento: Standard → Infrequent Access → Glacier (archival)

2. **Glue**: Servicio ETL serverless con PySpark
   - Auto-escalado de workers (DPU - Data Processing Units)
   - Glue Catalog: Metastore centralizado (Hive-compatible)
   - Crawlers: Descubrimiento automático de schemas

3. **Athena**: Motor SQL interactivo sobre S3
   - Basado en Presto, consulta Parquet/ORC/CSV sin infraestructura
   - Pricing: $5 por TB escaneado (optimizar con particiones)

4. **Redshift**: Data Warehouse MPP (Massively Parallel Processing)
   - Columnar storage con compresión
   - COPY desde S3, UNLOAD hacia S3

5. **EMR (Elastic MapReduce)**: Clusters Spark/Hadoop administrados
   - Spot instances para reducir costos 70-90%
   - Notebooks Jupyter integrados

**Arquitectura Lambda (Batch + Streaming):**
```
Fuentes → [Kinesis/Kafka] → S3 Raw → [Glue/EMR] → S3 Curated → [Athena/Redshift] → BI
                ↓
           Lambda (Near Real-Time)
```

**IAM Best Practices:**
- Roles > Users (principio de menor privilegio)
- Políticas: `s3:GetObject`, `s3:PutObject` (granular por bucket/prefix)
- No hardcodear credenciales: usar IAM Roles, AWS Secrets Manager

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. S3: Data Lake Básico

### 🗄️ **S3: Fundamentos del Data Lake**

**Conceptos Core:**

- **Bucket**: Contenedor global único (namespace global AWS)
  - Nombre: `mi-data-lake-123456` (minúsculas, números, guiones)
  - Región: `us-east-1`, `eu-west-1`, etc.

- **Prefix (Key)**: Jerarquía lógica simulada (no son carpetas reales)
  ```
  s3://bucket/raw/ventas/2025/10/ventas_2025_10_30.csv
           │     └─────┬─────┘ └──┬──┘ └────┬────┘
         Bucket      Prefix    Partición   Archivo
  ```

- **Versionamiento**: Protección contra eliminación accidental
  - Enabled: Cada PUT crea nueva versión con ID único
  - Lifecycle policies: Migrar versiones antiguas a Glacier

**Patrones de Organización:**

1. **Medallion Architecture (Databricks):**
   ```
   /bronze/    ← Datos crudos (raw, sin transformar)
   /silver/    ← Cleaned & validated
   /gold/      ← Business-level aggregates
   ```

2. **Por Dominio:**
   ```
   /sales/raw/, /sales/curated/
   /customers/raw/, /customers/curated/
   ```

3. **Particionamiento Hive-style:**
   ```
   /year=2025/month=10/day=30/data.parquet
   ```
   - Athena/Glue entienden automáticamente las particiones
   - Reduce escaneo en queries: `WHERE year=2025 AND month=10`

**Operaciones con boto3:**
- `put_object()`: Upload inline (para archivos pequeños <5GB)
- `upload_file()`: Upload desde disco local (multipart automático)
- `list_objects_v2()`: Listar con paginación (max 1000 por request)

**Costos:**
- PUT/COPY/POST/LIST: $0.005 per 1,000 requests
- GET/SELECT: $0.0004 per 1,000 requests
- Data transfer OUT: $0.09/GB (dentro de AWS: gratis)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
import os, json, io
import pandas as pd
# import boto3  # Descomenta si tienes credenciales configuradas

BUCKET = 'mi-data-lake-demo-123456'  # Cambia por un nombre único global
PREFIX_RAW = 'raw/ventas/'
PREFIX_CURATED = 'curated/ventas/'

print('👆 Define el bucket y prefijos antes de ejecutar contra AWS')

### 1.1 Crear bucket (opcional)

In [ ]:
# s3 = boto3.client('s3')
# try:
#     s3.create_bucket(Bucket=BUCKET, CreateBucketConfiguration={'LocationConstraint': 'us-east-1'})
#     print('✅ Bucket creado')
# except s3.exceptions.BucketAlreadyOwnedByYou:
#     print('ℹ️ Bucket ya existe')
# except Exception as e:
#     print('❌ Error creando bucket:', e)

### 1.2 Subir dataset de ejemplo a S3

In [ ]:
# df = pd.read_csv('../../datasets/raw/ventas.csv')
# csv_bytes = df.to_csv(index=False).encode('utf-8')
# s3.put_object(Bucket=BUCKET, Key=PREFIX_RAW + 'ventas_2025_10.csv', Body=csv_bytes)
# print('📤 Archivo subido a S3')

## 2. Glue: Transformaciones con PySpark (Job Script)

### 🔧 **AWS Glue: ETL Serverless con PySpark**

**Componentes de Glue:**

1. **Glue Data Catalog:**
   - Metastore centralizado (bases de datos + tablas)
   - Compatible con Hive Metastore (usado por Athena, EMR, Redshift Spectrum)
   - Schema discovery con Crawlers

2. **Glue Crawlers:**
   - Escanean S3 y detectan formato/schema automáticamente
   - Actualizan el Catalog con nuevas particiones
   - Schedule: Cron expressions (`cron(0 0 * * ? *)` = diario a medianoche)

3. **Glue Jobs (ETL Scripts):**
   - Python Shell (para scripts ligeros)
   - PySpark (para transformaciones distribuidas)
   - DPU (Data Processing Unit): 1 DPU = 4 vCPU + 16GB RAM
   - Pricing: $0.44 por DPU-hora

**GlueContext vs SparkContext:**
```python
# SparkContext: Estándar PySpark
sc = SparkContext()

# GlueContext: Extensión AWS con métodos adicionales
glueContext = GlueContext(sc)
glueContext.create_dynamic_frame_from_catalog()  # Lee desde Catalog
glueContext.write_dynamic_frame.from_options()    # Escribe con conversiones
```

**DynamicFrame vs DataFrame:**
- **DataFrame (Spark)**: Tipado estricto, requiere schema consistente
- **DynamicFrame (Glue)**: Schema flexible, maneja datos semi-estructurados
  - `resolveChoice()`: Resuelve conflictos de tipos
  - `unbox()`: Desempaqueta structs complejos

**Patrón de Job de Glue:**
```python
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'PARAM1'])
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# [ETL logic aquí]

job.commit()  # Marca job como exitoso en Catalog
```

**Optimizaciones:**
- **Pushdown Predicates**: Filtrar en lectura reduce datos procesados
- **Partition Pruning**: Leer solo particiones necesarias
- **Columnar Formats**: Parquet/ORC reducen IO 10x vs CSV

**Uso Real:**
Script Glue orquestado por Step Functions o EventBridge (CloudWatch Events) para procesamiento automático al detectar nuevos archivos en S3.

---
**Autor:** Luis J. Raigoso V. (LJRV)

Ejemplo de script de Glue (PySpark) para leer CSVs crudos de S3, limpiar/transformar y escribir en formato Parquet particionado por mes. Guarda este script como `glue_job.py` y súbelo a un Job de Glue.

In [ ]:
glue_job_script = r'''
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F

args = getResolvedOptions(sys.argv, ['JOB_NAME','BUCKET','PREFIX_RAW','PREFIX_CURATED'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

raw_path = f"s3://{args['BUCKET']}/{args['PREFIX_RAW']}*.csv"
curated_path = f"s3://{args['BUCKET']}/{args['PREFIX_CURATED']}"

# Leer CSV desde S3
df = spark.read.csv(raw_path, header=True, inferSchema=True)

# Transformaciones: limpiar nulos, filtrar valores positivos, agregar timestamp
df_clean = (df
    .dropna(subset=['venta_id', 'total'])
    .filter(F.col('total') > 0)
    .withColumn('procesado_en', F.current_timestamp())
)

# Escribir como Parquet particionado por mes
df_clean.write.partitionBy('fecha').mode('overwrite').parquet(curated_path)

job.commit()
print('✅ ETL completado')
'''
print('📝 Script de Glue listo para deployment')

## 3. AWS Lambda: Procesamiento Serverless Event-Driven

### ⚡ **AWS Lambda: Compute Serverless para Datos**

**¿Qué es Lambda?**

AWS Lambda es un servicio de computación serverless que ejecuta código en respuesta a eventos, sin necesidad de aprovisionar o administrar servidores.

**Características Principales:**

- **Event-Driven**: Se activa automáticamente con triggers (S3, DynamoDB, EventBridge, API Gateway)
- **Auto-Scaling**: Escala automáticamente de 0 a miles de invocaciones concurrentes
- **Pay-per-Use**: Cobro solo por tiempo de ejecución (milisegundos)
- **Límites**: Max 15 minutos por invocación, hasta 10GB de RAM

**Triggers Comunes para Data Engineering:**

1. **S3 Events**: Procesar archivos al subirse
   ```python
   # Lambda se activa cuando se crea objeto en S3
   def lambda_handler(event, context):
       bucket = event['Records'][0]['s3']['bucket']['name']
       key = event['Records'][0]['s3']['object']['key']
       # Procesar archivo
   ```

2. **EventBridge (CloudWatch Events)**: Cron jobs programados
   ```python
   # Ejecutar diariamente a las 2 AM
   # Regla EventBridge: rate(1 day) o cron(0 2 * * ? *)
   ```

3. **DynamoDB Streams**: Procesar cambios en tiempo real
   ```python
   # Lambda recibe cambios de DynamoDB
   for record in event['Records']:
       if record['eventName'] == 'INSERT':
           # Procesar nuevo registro
   ```

4. **SQS/SNS**: Cola de mensajes para procesamiento asíncrono
   ```python
   # Lambda consume mensajes de SQS automáticamente
   for record in event['Records']:
       body = json.loads(record['body'])
   ```

**Arquitectura Lambda para Datos:**

```
S3 (new file) → Lambda → Validar → S3 (curated)
                   ↓
                 SQS (errores) → Lambda (retry)
                   ↓
              CloudWatch Logs (monitoring)
```

**Capas (Layers):**
- Librerías compartidas (pandas, requests, etc.)
- Reduce tamaño del deployment package
- Reutilización entre funciones

**Best Practices:**
- **Idempotencia**: Lambda puede reintentar, asegurar que múltiples ejecuciones no causen problemas
- **Environment Variables**: Configuración (buckets, endpoints) sin hardcodear
- **Timeout**: Configurar según necesidad (default 3s, max 15min)
- **Memory**: Mayor memoria = más CPU (128MB a 10GB)
- **Dead Letter Queue (DLQ)**: SQS/SNS para errores no recuperables
- **Async invocation**: Para procesamiento no-blocking

**Limitaciones:**
- ❌ Max 15 minutos (para ETL largo, usar Glue/EMR)
- ❌ Max 10GB RAM
- ❌ Max 250MB deployment package (sin layers)
- ❌ /tmp storage max 512MB (efímero)
- ✅ Ideal para: Validación, transformaciones ligeras, triggers

**Pricing:**
- Requests: $0.20 por 1M requests
- Duration: $0.0000166667 por GB-segundo
- Ejemplo: 1M requests x 1GB x 1s = ~$17/mes
- Free tier: 1M requests + 400,000 GB-segundos/mes

**Alternativas:**
- **Glue Python Shell**: Para scripts Python no-PySpark (no límite 15min)
- **ECS Fargate**: Para procesos largos sin administrar servidores
- **Step Functions**: Orquestación de múltiples Lambdas

---
**Autor:** Luis J. Raigoso V. (LJRV)

### 3.1 Ejemplo: Lambda para validar CSV al subir a S3

In [ ]:
lambda_handler_code = '''
import json
import boto3
import pandas as pd
from io import BytesIO

s3 = boto3.client('s3')

def lambda_handler(event, context):
    """
    Lambda function triggered by S3 PutObject event.
    Validates CSV structure and moves to curated zone if valid.
    """
    # Extraer información del evento
    for record in event['Records']:
        bucket = record['s3']['bucket']['name']
        key = record['s3']['object']['key']
        
        print(f'Processing file: s3://{bucket}/{key}')
        
        # Skip if not CSV
        if not key.endswith('.csv'):
            print('Skipping non-CSV file')
            continue
        
        try:
            # Leer CSV desde S3
            response = s3.get_object(Bucket=bucket, Key=key)
            csv_content = response['Body'].read()
            df = pd.read_csv(BytesIO(csv_content))
            
            # Validaciones
            required_cols = ['venta_id', 'cliente_id', 'total']
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if missing_cols:
                raise ValueError(f'Missing columns: {missing_cols}')
            
            # Validar tipos y valores
            if df['total'].isnull().sum() > 0:
                raise ValueError(f'Null values found in total column: {df["total"].isnull().sum()}')
            
            if (df['total'] < 0).sum() > 0:
                raise ValueError(f'Negative values in total: {(df["total"] < 0).sum()}')
            
            # Si válido, mover a zona curated
            curated_key = key.replace('/raw/', '/curated/')
            s3.copy_object(
                Bucket=bucket,
                CopySource={'Bucket': bucket, 'Key': key},
                Key=curated_key
            )
            
            print(f'✅ File validated and copied to: s3://{bucket}/{curated_key}')
            
            return {
                'statusCode': 200,
                'body': json.dumps({
                    'message': 'File processed successfully',
                    'rows': len(df),
                    'output': f's3://{bucket}/{curated_key}'
                })
            }
            
        except Exception as e:
            print(f'❌ Error processing file: {str(e)}')
            
            # Mover a carpeta de errores
            error_key = key.replace('/raw/', '/errors/')
            s3.copy_object(
                Bucket=bucket,
                CopySource={'Bucket': bucket, 'Key': key},
                Key=error_key
            )
            
            # Enviar a DLQ o SNS para alertas
            return {
                'statusCode': 500,
                'body': json.dumps({
                    'error': str(e),
                    'file': f's3://{bucket}/{key}'
                })
            }
'''

print('📝 Lambda handler para validación de CSV')
print('\\nPara deployar:')
print('1. Crear función Lambda en AWS Console')
print('2. Agregar layer con pandas (o usar Lambda Container)')
print('3. Configurar trigger S3 en bucket/raw/')
print('4. Ajustar timeout (60s) y memoria (512MB)')

## 4. Athena: SQL sobre S3 sin Infraestructura

### 📊 **Amazon Athena: Query Engine Serverless**

Athena permite ejecutar queries SQL estándar directamente sobre datos en S3 sin necesidad de cargarlos en una base de datos.

**Características:**
- **Basado en Presto/Trino**: Engine SQL distribuido
- **Sin servidores**: No hay clusters que administrar
- **Standard SQL**: Compatible con ANSI SQL
- **Formatos soportados**: CSV, JSON, Parquet, ORC, Avro

**Pricing:** $5 por TB de datos escaneados

**Optimizaciones clave:**
1. **Particionar datos**: Reduce escaneo con `WHERE year=2025 AND month=10`
2. **Formato columnar**: Parquet/ORC reducen escaneo 10x vs CSV
3. **Comprimir**: SNAPPY, GZIP, ZSTD reducen tamaño
4. **Proyección**: `SELECT col1, col2` en vez de `SELECT *`

**Ejemplo de uso:**

```sql
-- Crear tabla externa apuntando a S3
CREATE EXTERNAL TABLE ventas (
    venta_id INT,
    cliente_id INT,
    producto_id INT,
    cantidad INT,
    total DECIMAL(10,2),
    fecha DATE
)
PARTITIONED BY (year INT, month INT)
STORED AS PARQUET
LOCATION 's3://mi-data-lake/curated/ventas/';

-- Agregar particiones
MSCK REPAIR TABLE ventas;

-- Query con partition pruning (solo escanea Oct 2025)
SELECT cliente_id, SUM(total) as total_ventas
FROM ventas
WHERE year = 2025 AND month = 10
GROUP BY cliente_id
ORDER BY total_ventas DESC
LIMIT 10;
```

**Integración con Python (boto3):**

```python
import boto3
import time

athena = boto3.client('athena')

query = \"\"\"
SELECT cliente_id, COUNT(*) as num_ventas
FROM ventas
WHERE year = 2025 AND month = 10
GROUP BY cliente_id
\"\"\"

response = athena.start_query_execution(
    QueryString=query,
    QueryExecutionContext={'Database': 'mi_database'},
    ResultConfiguration={'OutputLocation': 's3://mi-bucket/athena-results/'}
)

query_id = response['QueryExecutionId']

# Esperar a que termine
while True:
    status = athena.get_query_execution(QueryExecutionId=query_id)
    state = status['QueryExecution']['Status']['State']
    
    if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
        break
    time.sleep(2)

if state == 'SUCCEEDED':
    results = athena.get_query_results(QueryExecutionId=query_id)
    # Procesar resultados
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 5. Arquitectura Completa: S3 + Lambda + Glue + Athena

### 🏗️ **Pipeline End-to-End en AWS**

**Flujo completo de datos:**

```
1. INGESTA
   Fuentes → S3 (bucket/raw/)
   
2. VALIDACIÓN (Lambda)
   S3 Event → Lambda → Validar schema
                ↓
           S3 (curated/) o S3 (errors/)
   
3. TRANSFORMACIÓN (Glue)
   EventBridge (cron) → Glue Job (PySpark)
                          ↓
                    S3 (gold/) Parquet particionado
   
4. CATALOGACIÓN (Glue Crawler)
   Glue Crawler → Detecta schema → Glue Catalog
   
5. ANÁLISIS (Athena)
   Athena → SQL queries → S3 (resultados)
            ↓
       QuickSight (BI) / Python (análisis)
```

**Ejemplo de implementación:**

**Paso 1: Configurar S3 Bucket**
```bash
aws s3 mb s3://mi-data-lake-2025
aws s3api put-bucket-versioning \\
    --bucket mi-data-lake-2025 \\
    --versioning-configuration Status=Enabled
```

**Paso 2: Deploy Lambda**
```bash
# Empaquetar código con dependencias
zip lambda_function.zip lambda_function.py
aws lambda create-function \\
    --function-name validate-csv \\
    --runtime python3.10 \\
    --role arn:aws:iam::123456:role/lambda-s3-role \\
    --handler lambda_function.lambda_handler \\
    --zip-file fileb://lambda_function.zip \\
    --timeout 60 \\
    --memory-size 512
```

**Paso 3: Configurar S3 Event Notification**
```json
{
  "LambdaFunctionConfigurations": [
    {
      "LambdaFunctionArn": "arn:aws:lambda:us-east-1:123456:function:validate-csv",
      "Events": ["s3:ObjectCreated:*"],
      "Filter": {
        "Key": {
          "FilterRules": [
            {"Name": "prefix", "Value": "raw/"},
            {"Name": "suffix", "Value": ".csv"}
          ]
        }
      }
    }
  ]
}
```

**Paso 4: Crear Glue Job**
```bash
aws glue create-job \\
    --name transform-ventas \\
    --role AWSGlueServiceRole \\
    --command Name=glueetl,ScriptLocation=s3://scripts/glue_job.py \\
    --default-arguments '{
        "--BUCKET":"mi-data-lake-2025",
        "--PREFIX_RAW":"curated/ventas/",
        "--PREFIX_CURATED":"gold/ventas/"
    }' \\
    --glue-version 3.0 \\
    --number-of-workers 2 \\
    --worker-type G.1X
```

**Paso 5: Programar Glue Job con EventBridge**
```bash
aws events put-rule \\
    --name daily-etl \\
    --schedule-expression "cron(0 2 * * ? *)"

aws events put-targets \\
    --rule daily-etl \\
    --targets "Id"="1","Arn"="arn:aws:glue:us-east-1:123456:job/transform-ventas"
```

**Paso 6: Query con Athena**
```python
import boto3

athena = boto3.client('athena')

response = athena.start_query_execution(
    QueryString='SELECT * FROM ventas WHERE year=2025 LIMIT 10',
    QueryExecutionContext={'Database': 'mi_database'},
    ResultConfiguration={'OutputLocation': 's3://mi-data-lake-2025/athena-results/'}
)
```

**Monitoring:**
- **CloudWatch Logs**: Lambda + Glue logs
- **CloudWatch Metrics**: Invocations, errors, duration
- **X-Ray**: Tracing distribuido
- **CloudTrail**: Auditoría de acciones

**Costos estimados (ejemplo):**
- S3 storage (100GB): ~$2.30/mes
- Lambda (10M invocations, 512MB, 1s avg): ~$17/mes
- Glue Job (diario, 2 DPU, 30 min): ~$13/mes
- Athena (10TB escaneados/mes): ~$50/mes
- **Total**: ~$82/mes

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 6. Conclusión y Mejores Prácticas

### 🎯 **Key Takeaways de AWS para Data Engineering**

**Servicios Clave:**
- **S3**: Foundation del data lake (durabilidad 11 nines)
- **Lambda**: Validación y transformaciones ligeras event-driven
- **Glue**: ETL serverless con PySpark para transformaciones pesadas
- **Athena**: Queries SQL ad-hoc sobre S3 sin infraestructura
- **Redshift**: Data Warehouse para analytics de alta performance
- **EMR**: Clusters Spark/Hadoop para procesamiento masivo

**Cuándo usar cada servicio:**

| Caso de Uso | Servicio | Razón |
|-------------|----------|-------|
| Transformación <5 min, <10GB RAM | **Lambda** | Serverless, rápido, económico |
| ETL batch, PySpark distribuido | **Glue** | Auto-scaling, sin servidores |
| Queries exploratorias sobre S3 | **Athena** | Sin setup, paga por escaneo |
| DW con TBs, queries complejas | **Redshift** | MPP, optimizado para analytics |
| Custom Spark jobs, ML training | **EMR** | Control total, spot instances |
| Real-time streaming | **Kinesis** | Managed Kafka-like service |

**Best Practices:**

1. **Organización de S3**:
   - Usar particionamiento Hive-style (`year=2025/month=10/day=30/`)
   - Formatos columnares (Parquet/ORC) para reducir costos Athena
   - Lifecycle policies para migrar a Glacier

2. **Lambda**:
   - Idempotencia (múltiples invocaciones no dañan)
   - DLQ para manejar errores
   - Environment variables para configuración
   - Layers para compartir dependencias

3. **Glue**:
   - DynamicFrame para datos semi-estructurados
   - Glue Catalog como metastore centralizado
   - Partition pruning para optimizar lectura
   - Bookmark para procesamiento incremental

4. **Athena**:
   - Particionar datos agresivamente
   - CTAS (Create Table As Select) para materializar resultados
   - Workgroups para cost control por equipo

5. **Seguridad**:
   - IAM Roles (nunca hardcodear credenciales)
   - S3 bucket policies (restringir por prefix)
   - VPC endpoints para tráfico privado
   - Encryption at-rest (S3-SSE) y in-transit (HTTPS)

6. **Costos**:
   - Spot instances para EMR (70-90% descuento)
   - Reserved capacity para Glue/Redshift predictibles
   - Comprimir datos (reduce storage + transfer)
   - Monitoreo con AWS Cost Explorer

**Próximos Pasos:**
1. ✅ Completar notebooks GCP y Azure para comparación
2. ✅ Practicar con [AWS Free Tier](https://aws.amazon.com/free/)
3. ✅ Certificación: [AWS Certified Data Analytics - Specialty](https://aws.amazon.com/certification/certified-data-analytics-specialty/)
4. ✅ Explorar Step Functions para orquestación compleja

**Happy data engineering en AWS! 🚀**

---
**Autor Final:** LuisRai (Luis J. Raigoso V.)  
© 2024-2025 - Data Engineering Modular Course

---

## 🧭 Navegación

**← Anterior:** [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)

**Siguiente →:** [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer →](03b_cloud_gcp.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb) ← 🔵 Estás aquí
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb)
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb)
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb)
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
